In [22]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import os


loading the 6 datasets which includes the crisis and the corresponding total coverage received over the mentioned crisis days , the table also includes coverage quantitative;ly calculated per day,per required_funding,per people_effected

In [26]:
csv_dir = "../data/raw/"  

crises_df = pd.read_csv(os.path.join(csv_dir, "chart1_overall_coverage_bar.csv"))
monthly_df = pd.read_csv(os.path.join(csv_dir, "chart3_monthly_coverage.csv"))
outlet_df = pd.read_csv(os.path.join(csv_dir, "chart4_crisis_coverage_per_outlet_chart.csv"))
framing_df = pd.read_csv(os.path.join(csv_dir, "framing_per_article.csv"))
sentiment_df = pd.read_csv(os.path.join(csv_dir, "sentiment_per_article.csv"))
victim_df = pd.read_csv(os.path.join(csv_dir, "victim_causor_per_article.csv"))

Assign crisis_id to the data frames

In [27]:
crises_df = crises_df.reset_index().rename(columns={"index":"crisis_id"})
crises_df["crisis_id"] = crises_df["crisis_id"] + 1  # Start IDs at 1

Create a mapping: crisis_name -> crisis_id

In [28]:

crisis_map = dict(zip(crises_df["crisis_name"], crises_df["crisis_id"]))

Adding crisis_id to child tables

In [29]:
monthly_df["crisis_id"] = monthly_df["crisis_name"].map(crisis_map)
outlet_df["crisis_id"] = outlet_df["crisis_name"].map(crisis_map)
framing_df["crisis_id"] = framing_df["crisis_name"].map(crisis_map)
sentiment_df["crisis_id"] = sentiment_df["crisis_name"].map(crisis_map)
victim_df["crisis_id"] = victim_df["crisis_name"].map(crisis_map)

try create SQLite DB

In [30]:
db_file = "humanitarian.db"
conn = sqlite3.connect(db_file)
cursor = conn.cursor()


To Run schema.sql

In [38]:
with open("C:\\Users\\nikhi\\OneDrive\\Documents\\Data science project\\IDS_Project\\sql\\schema.sql", "r") as f:
    cursor.executescript(f.read())

populating tables:

# Crises table

In [39]:
crises_df[['crisis_id','crisis_name','start_date','fund_required','people_affected','crisis_days',
           'raw_coverage','coverage_per_day','coverage_per_funding','coverage_per_people']].to_sql(
    "crises", conn, if_exists="append", index=False)

10

# Monthly coverage table

In [41]:
monthly_df[['monthly_id','crisis_id','year_month','coverage_count']] = monthly_df.reset_index()
monthly_df.to_sql(
    "monthly_coverage",
    conn,
    if_exists="append",
    index=False
)

ValueError: Columns must be same length as key

In [ ]:
plt.figure(figsize=(10,6))
coverage.sort_values("coverage_per_day", ascending=False).plot(
    x="crisis_name",
    y="coverage_per_day",
    kind="bar",
    legend=False
)
plt.title("Coverage Per Day by Crisis")
plt.ylabel("Coverage Per Day")
plt.xticks(rotation=90)
plt.show()


In [ ]:
plt.figure(figsize=(10,6))

sns.scatterplot(
    data=coverage,
    x="crisis_name",
    y="coverage_per_day"
)

plt.xticks(rotation=90)  
plt.title("Coverage per Day by Crisis")
plt.xlabel("Crisis Name")
plt.ylabel("Coverage per Day")

plt.tight_layout()
plt.show()


In [ ]:
conn = sqlite3.connect("../data/project.db")

coverage.to_sql("crises", conn, if_exists="replace", index=False)

conn.close()
